In [1]:
import sys
import os, copy
import torch
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import OrderedDict
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


#### Model conversion utils

In [2]:
def state_dict_to_vector(state_dict, remove_keys=[]):
    shared_state_dict = copy.deepcopy(state_dict)
    for key in remove_keys:
        if key in shared_state_dict:
            del shared_state_dict[key]
    sorted_shared_state_dict = OrderedDict(sorted(shared_state_dict.items()))
    return torch.nn.utils.parameters_to_vector(
        [value.reshape(-1) for key, value in sorted_shared_state_dict.items()]
    )


def vector_to_state_dict(vector, state_dict, remove_keys=[]):
    # create a reference dict to define the order of the vector
    reference_dict = copy.deepcopy(state_dict)
    for key in remove_keys:
        if key in reference_dict:
            del reference_dict[key]
    sorted_reference_dict = OrderedDict(sorted(reference_dict.items()))

    # create a shared state dict using the refence dict
    torch.nn.utils.vector_to_parameters(vector, sorted_reference_dict.values())

    # add back the encoder and decoder embedding weights.
    if "transformer.shared.weight" in sorted_reference_dict:
        for key in remove_keys:
            sorted_reference_dict[key] = sorted_reference_dict[
                "transformer.shared.weight"
            ]
    return sorted_reference_dict


def add_ptm_to_tv(tv_dict, ptm_dict):
    assert set(tv_dict.keys()) == set(
        ptm_dict.keys()
    ), "Differing parameter names in models."
    final_dict = copy.deepcopy(tv_dict)
    for k, v in ptm_dict.items():
        final_dict[k] = tv_dict[k] + v
    return final_dict


def check_parameterNamesMatch(checkpoints):
    parameter_names = set(checkpoints[0].keys())

    if len(checkpoints) >= 2:
        # raise ValueError("Number of models is less than 2.")
        for checkpoint in checkpoints[1:]:
            current_parameterNames = set(checkpoint.keys())
            if current_parameterNames != parameter_names:
                raise ValueError(
                    "Differing parameter names in models. "
                    f"The different parameters are {parameter_names.symmetric_difference(current_parameterNames)}"
                )

def check_state_dicts_equal(state_dict1, state_dict2):
    if set(state_dict1.keys()) != set(state_dict2.keys()):
        return False

    for key in state_dict1.keys():
        if not torch.equal(state_dict1[key], state_dict2[key]):
            return False

    return True

#### Load base model and the Finetuned Models to merge

In [3]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base").to("cpu").state_dict()

# some keys are tied together so they need to be removed,
# for example in this case, we will only keep the shared.weight and remove the other two.
assert (model['shared.weight'] - model['encoder.embed_tokens.weight']).sum() == 0
assert (model['shared.weight'] - model['decoder.embed_tokens.weight']).sum() == 0

# Load all the models to merge
model_rte = AutoModelForSeq2SeqLM.from_pretrained("PavanNeerudu/t5-base-finetuned-rte").to("cpu").state_dict()
model_mnli = AutoModelForSeq2SeqLM.from_pretrained("PavanNeerudu/t5-base-finetuned-mnli").to("cpu").state_dict()
model_sst2 = AutoModelForSeq2SeqLM.from_pretrained("PavanNeerudu/t5-base-finetuned-sst2").to("cpu").state_dict()

/playpen/prateek/anaconda3/envs/geocl/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [4]:
ft_checks = [model_rte, model_mnli, model_sst2]
ptm_check = model

# check if all checkpoints have the same paramters.
check_parameterNamesMatch(ft_checks + [ptm_check])

# Removing the two keys from state dict when creating the task vector.
# Basically these keys are not involved in the global operations like the computation of topk.
remove_keys = [
    "transformer.encoder.embed_tokens.weight",
    "transformer.decoder.embed_tokens.weight",
]

print(f"Flattening out Checkpoints")
flat_ft = torch.vstack(
    [state_dict_to_vector(check, remove_keys) for check in ft_checks]
)
flat_ptm = state_dict_to_vector(ptm_check, remove_keys)

# Creating Task vectors
tv_flat_checks = flat_ft - flat_ptm

# check if the vectorized state dicts can be converted back to the original state dicts
# covnert back the flat task vectors to state dict and see if the original and converted sd's are equal
assert check_state_dicts_equal(
        vector_to_state_dict(flat_ptm, ptm_check, remove_keys), ptm_check
    )
assert all(
    [
        check_state_dicts_equal(
            vector_to_state_dict(flat_ft[i], ptm_check, remove_keys), ft_checks[i]
        )
        for i in range(len(ft_checks))
    ]
)

Flattening out Checkpoints


#### Merge Utils

In [5]:
## TIES MERGING UTILS

def topk_values_mask(M, K=0.7, return_mask=False):
    if K > 1:
        K /= 100

    original_shape = M.shape
    if M.dim() == 1:
        M = M.unsqueeze(0)

    n, d = M.shape
    k = int(d * K)
    k = d - k  # Keep top k elements instead of bottom k elements

    # Find the k-th smallest element by magnitude for each row
    kth_values, _ = M.abs().kthvalue(k, dim=1, keepdim=True)
    # Create a mask tensor with True for the top k elements in each row
    mask = M.abs() >= kth_values
    final_mask = mask.squeeze() if original_shape == M.squeeze().shape else mask

    if return_mask:
        return M * final_mask, final_mask.float().mean(dim=1), final_mask
    return M * final_mask, final_mask.float().mean(dim=1)


def resolve_zero_signs(sign_to_mult, method="majority"):
    majority_sign = torch.sign(sign_to_mult.sum())

    if method == "majority":
        sign_to_mult[sign_to_mult == 0] = majority_sign
    elif method == "minority":
        sign_to_mult[sign_to_mult == 0] = -1 * majority_sign
    return sign_to_mult


def resolve_sign(Tensor):
    sign_to_mult = torch.sign(Tensor.sum(dim=0))
    sign_to_mult = resolve_zero_signs(sign_to_mult, "majority")
    return sign_to_mult


def disjoint_merge(Tensor, merge_func, sign_to_mult):

    merge_func = merge_func.split("-")[-1]

    # If sign is provided then we select the corresponding entries and aggregate.
    if sign_to_mult is not None:
        rows_to_keep = torch.where(
            sign_to_mult.unsqueeze(0) > 0, Tensor > 0, Tensor < 0
        )
        selected_entries = Tensor * rows_to_keep
    # Else we select all non-zero entries and aggregate.
    else:
        rows_to_keep = Tensor != 0
        selected_entries = Tensor * rows_to_keep

    if merge_func == "mean":
        non_zero_counts = (selected_entries != 0).sum(dim=0).float()
        disjoint_aggs = torch.sum(selected_entries, dim=0) / torch.clamp(
            non_zero_counts, min=1
        )
    elif merge_func == "sum":
        disjoint_aggs = torch.sum(selected_entries, dim=0)
    elif merge_func == "max":
        disjoint_aggs = selected_entries.abs().max(dim=0)[0]
        disjoint_aggs *= sign_to_mult
    else:
        raise ValueError(f"Merge method {merge_func} is not defined.")

    return disjoint_aggs


def ties_merging(
    flat_task_checks,
    reset_thresh=None,
    merge_func="",
):
    all_checks = flat_task_checks.clone()
    updated_checks, *_ = topk_values_mask(
        all_checks, K=reset_thresh, return_mask=False
    )
    print(f"RESOLVING SIGN")
    final_signs = resolve_sign(updated_checks)
    assert final_signs is not None
    
    print(f"Disjoint AGGREGATION: {merge_func}")
    merged_tv = disjoint_merge(updated_checks, merge_func, final_signs)
    
    return merged_tv



#### TIES Merging

In [6]:
# TIES Merging example
K = 20
merge_func = "dis-mean"
lamda = 1

# return merged flat task vector
merged_tv = ties_merging(
    tv_flat_checks,
    reset_thresh=K,
    merge_func=merge_func,
)

# add back the PTM to the flat merged task vector
merged_check = flat_ptm + lamda * merged_tv

# convert the flat merged checkpoint to a state dict
merged_state_dict = vector_to_state_dict(
    merged_check, ptm_check, remove_keys=remove_keys
)

RESOLVING SIGN
Disjoint AGGREGATION: dis-mean


#### Task Vector Merging

In [7]:
## TASK VECTOR MERGING UTILS

def aggregate(T, agg_type, dim=0):
    if agg_type == "mean":
        result = torch.mean(T, dim=dim)
    elif agg_type == "sum":
        result = torch.sum(T, dim=dim)
    else:
        raise ValueError("Invalid agg_type: %s" % agg_type)

    return result

def tv_merging(tv_flat_checks):
    """Merging by creating and scaling Task Vectors"""
    all_checks = tv_flat_checks.clone()
    tv_merged_check = aggregate(all_checks, "sum")
    return tv_merged_check


In [8]:
# Task Vector Merging example
lamda = 0.4

merged_tv = tv_merging(tv_flat_checks)
merged_check = flat_ptm + lamda * merged_tv
merged_state_dict = vector_to_state_dict(
    merged_check, ptm_check, remove_keys=remove_keys
)